This code is modified from the original code base: https://github.com/Natooz/MidiTok \\
Original Author: 2021 Nathan Fradet


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
gpu_info = ! nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Mon May  8 18:36:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('Not using a high-RAM runtime')
else:
    print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
!pip install pretty_midi
!pip install unidecode
!pip install miditok
!pip install tokenizers

!pip install torchtoolkit
!pip install transformers
!pip install evaluate
!pip install miditoolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%cd /content/drive/MyDrive/new_music

/content/drive/MyDrive/new_music


In [5]:
import torch
from load_data import make_meta_data, MIDIDataset, DataCollatorGen, make_meta_data_pop
from model_transformer import model
from generate import collate_gen_left, generate_batch
from transformers import Trainer, TrainingArguments
import pandas as pd
from miditok.constants import CHORD_MAPS
from miditok import REMI, MIDITokenizer
from torchtoolkit.data import create_subsets
from pathlib import Path
from torch.utils.data import DataLoader

In [7]:
midi_root_dir = '/content/drive/MyDrive/theme_transformer/new_test/pop909/POP909/'
# midi_root_dir = '/content/drive/MyDrive/theme_transformer/midi_data/midis/'
midi_meta_dict = make_meta_data_pop(midi_root_dir)
# midi_meta_dict = make_meta_data(midi_root_dir)
df = pd.DataFrame(midi_meta_dict)
print(df['path'])

0      /content/drive/MyDrive/theme_transformer/new_t...
1      /content/drive/MyDrive/theme_transformer/new_t...
2      /content/drive/MyDrive/theme_transformer/new_t...
3      /content/drive/MyDrive/theme_transformer/new_t...
4      /content/drive/MyDrive/theme_transformer/new_t...
                             ...                        
904    /content/drive/MyDrive/theme_transformer/new_t...
905    /content/drive/MyDrive/theme_transformer/new_t...
906    /content/drive/MyDrive/theme_transformer/new_t...
907    /content/drive/MyDrive/theme_transformer/new_t...
908    /content/drive/MyDrive/theme_transformer/new_t...
Name: path, Length: 909, dtype: object


Load and Tokenize Data

In [8]:
pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': True,
                     'rest_range': (2, 8),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (40, 250),  # (min, max)
                     'Program': False,
                     "chord_maps": CHORD_MAPS,
                     "chord_tokens_with_root_note": True,
                     "chord_unknown": False}
special_tokens = ["PAD", "BOS", "EOS"]

# Creates the tokenizer convert MIDIs to tokens
tokens_path = '/content/drive/MyDrive/new_music/json_no_bpe_pop'
tokenizer = REMI(pitch_range, beat_res, nb_velocities, additional_tokens, special_tokens=special_tokens)

tokenizer.tokenize_midi_dataset(list(df['path']), tokens_path)

tokens_bpe_path = Path('/content/drive/MyDrive/new_music/json_bpe_pop')
tokens_bpe_path.mkdir(exist_ok=True, parents=True)
tokenizer.learn_bpe(
    vocab_size=500, # Encode the first 500 music events
    tokens_paths=list(Path(tokens_path).glob("**/*.json")),
    start_from_empty_voc=False,
)
tokenizer.apply_bpe_to_dataset(
    tokens_path,
    tokens_bpe_path,
)


tokens_paths = list(tokens_bpe_path.glob("**/*.json"))
dataset = MIDIDataset(
    tokens_paths, max_seq_len=512, min_seq_len=450, 
)
subset_train, subset_valid = create_subsets(dataset, [0.1])

Applying BPE to dataset: 100%|██████████| 909/909 [00:15<00:00, 58.16it/s]
Loading data: /content/drive/MyDrive/new_music/json_bpe_pop: 100%|██████████| 909/909 [00:01<00:00, 546.55it/s]


In [9]:
print(len(tokenizer))
print(tokenizer['PAD_None'])
print(tokenizer['BOS_None'])
print(tokenizer['EOS_None'])

500
0
1
2


In [11]:
tokenizer_path = '/content/drive/MyDrive/new_music/Remi_tokenizer/tokenizer_pop.json'
# tokenizer_path = '/content/drive/MyDrive/new_music/Remi_tokenizer/tokenizer.json'
tokenizer.save_params(Path(tokenizer_path))

In [12]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(500, 512)
    (wpe): Embedding(2048, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-3): 4 x GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=500, bias=False)
)


Train

In [13]:
for param in model.parameters():
    param.requires_grad = True

# class RegressionTrainer(Trainer):

#         def compute_loss(self,
#                          model,
#                          inputs,
#                          return_outputs=False):
#             labels = inputs.get("labels").float()
#             outputs = model(**inputs)
#             logits = outputs.get('logits')
#             pred_ids = argmax(logits, dim=-1).float()
#             pred_ids.requires_grad = True
#             labels.requires_grad = True
#             loss = torch.nn.CrossEntropyLoss()(pred_ids.view(-1), labels.view(-1).float())
#             print(loss.requires_grad)
#             return (loss, outputs) if return_outputs else loss

save_dir = '/content/drive/MyDrive/new_music/experiments_pop'

training_config = TrainingArguments(
    save_dir, False, True, True, False, "steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=48,
    gradient_accumulation_steps=3,
    eval_accumulation_steps=None,
    eval_steps=100,
    learning_rate=1e-4,
    max_steps=4000,
    log_level="debug",
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=5,
    fp16=True,
    load_best_model_at_end=True,
    optim="adamw_torch",
    report_to=["tensorboard"],
    gradient_checkpointing=True,
    label_names=None,
)

trainer = Trainer(
    model=model,
    args=training_config,
    data_collator=DataCollatorGen(tokenizer["PAD_None"]),
    train_dataset=subset_train,
    eval_dataset=subset_valid,
    callbacks=None,
)

# Training
train_result = trainer.train()
trainer.save_model(save_dir)
trainer.log_metrics(save_dir, train_result.metrics)
trainer.save_metrics(save_dir, train_result.metrics)
trainer.save_state()

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend
***** Running training *****
  Num examples = 1,881
  Num Epochs = 103
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 3
  Total optimization steps = 4,000
  Number of trainable parameters = 13,915,136
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: file_labels. If file_labels are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
100,2.575800,2.480050
200,2.271900,2.251262
300,2.161600,2.175411
400,2.090900,2.114170
500,1.970000,1.941209
600,1.764500,1.725274
700,1.650200,1.639103
800,1.576300,1.592391
900,1.539700,1.564583
1000,1.497400,1.546894


***** Running Evaluation *****
  Num examples = 208
  Batch size = 48
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: file_labels. If file_labels are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 208
  Batch size = 48
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: file_labels. If file_labels are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 208
  Batch size = 48
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: file_labels. If file_labels are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 208
  Batc

***** /content/drive/MyDrive/new_music/experiments_pop metrics *****
  epoch                    =     101.69
  total_flos               =  6901679GF
  train_loss               =     1.3309
  train_runtime            = 0:07:33.97
  train_samples_per_second =    422.935
  train_steps_per_second   =      8.811


In [ ]:
model_weights = torch.load('/content/drive/MyDrive/new_music/experiments/checkpoint-4000/pytorch_model.bin')
model.load_state_dict(model_weights)

<All keys matched successfully>

Generate Results

In [14]:
gen_results_path = '/content/drive/MyDrive/new_music/results_pop/'
dataloader_test = DataLoader(subset_valid, batch_size=16, collate_fn=collate_gen_left, shuffle=True)
generate_batch(dataloader_test, model, 'cuda', tokenizer, gen_results_path)

Testing model / Generating results: 100%|██████████| 13/13 [00:56<00:00,  4.35s/it]


In [ ]:
del model

In [ ]:
print(model)

NameError: ignored